In [1]:
import re
import io
import time
import joblib
import requests
from tqdm import tqdm
import pandas as pd
import pubchempy as pcp
from Bio.KEGG import REST
from bioservices import KEGG, ChEBI

In [2]:
df = pd.read_csv('test.csv')

In [12]:
smiles_list = df.Compound.tolist()

with open('kegg_ids.csv', 'a') as output_file:
    output_file.write('comp_id,score,pwy_id\n')
    for i, smiles in tqdm(enumerate(smiles_list), total=len(smiles_list)):
        data = {
            'smiles': smiles,
            'cutoff': 0.01,
            'limit': 3
        }
        
        response = requests.post('http://rest.genome.jp/simcomp/', data=data)
        
        if response.status_code == 200:
            resp = '\n'.join([r + f',{i}' for r in response.text.replace('\t', ',').split('\n')[:-1]])
            output_file.write(f'{resp}\n')
        else:
            continue

        time.sleep(3)

100%|███████████████████████████████████████| 201/201 [1:20:40<00:00, 24.08s/it]


In [13]:
def to_df(result):
    return pd.read_table(io.StringIO(result), header=None)

In [14]:
kegg_df = pd.read_csv('kegg_ids.csv')
kegg_ids = kegg_df.comp_id.unique()

In [15]:
reaction_ids = pd.DataFrame()
for idx in tqdm(kegg_ids):
    try:
        result = REST.kegg_link("rn", idx).read()
        reaction_ids = pd.concat([reaction_ids, to_df(result)], axis=0)
    except:
        continue

    time.sleep(3)

reaction_ids.columns = ['comp_id', 'rn_id']
reaction_ids.to_csv('reaction_ids.csv', index=False)

100%|█████████████████████████████████████████| 992/992 [41:47<00:00,  2.53s/it]


In [16]:
reactions = []
for idx in tqdm(reaction_ids.rn_id.tolist()):
    try:
        result = REST.kegg_get(idx).read()
        reactions.extend([row for row in result.split('\n') if 'EQUATION' in row])
    except:
        continue

    time.sleep(3)

joblib.dump(reactions, 'reactions.pkl')

100%|█████████████████████████████████████| 3398/3398 [3:48:39<00:00,  4.04s/it]


TypeError: cannot pickle 'generator' object

In [36]:
rs2ps = []
for r in reactions:
    rs2ps.append([re.findall(r'C\d{5}', s) for s in r.split('<=>')])

In [68]:
pathways = []
for rs,ps in rs2ps:
    [[pathways.append([r, p]) for p in ps] for r in rs]
pathways = pd.DataFrame(pathways, columns = ['Reactant', 'Product'])
pathways = pathways.drop_duplicates().reset_index(drop=True)
pathways.to_csv('kegg_pathways.csv', index=False)

In [69]:
unique_kegg_ids = set(pathways.Reactant.tolist() + pathways.Product.tolist())

In [70]:
kegg2smiles = {}
for idx in tqdm(unique_kegg_ids):
    try:
        kegg_con = KEGG()
        kegg_entry = kegg_con.parse(kegg_con.get(idx))

        chebi_con = ChEBI()
        chebi_entry = chebi_con.getCompleteEntity('CHEBI:' + kegg_entry['DBLINKS']['ChEBI'])

        kegg2smiles[idx] = chebi_entry.smiles
    except:
        continue
    
    time.sleep(3)

100%|█████████████████████████████████████| 3151/3151 [2:53:57<00:00,  3.31s/it]


In [281]:
pathways['Reactant'] = pathways['Reactant'].replace(kegg2smiles)
pathways['Product'] = pathways['Product'].replace(kegg2smiles)

In [282]:
missing_ids = re.findall(r'C\d{5}', ' '.join(set(pathways.Reactant.tolist() + pathways.Product.tolist())))

In [283]:
len(missing_ids)

68

In [95]:
for idx in tqdm(missing_ids):
    try:
        kegg_con = KEGG()
        kegg_entry = kegg_con.parse(kegg_con.get(idx))

        smiles = pcp.Compound.from_cid(kegg_entry['DBLINKS']['PubChem']).isomeric_smiles

        kegg2smiles[idx] = smiles
    except:
        continue
    
    time.sleep(3)

100%|█████████████████████████████████████████| 425/425 [27:38<00:00,  3.90s/it]


In [280]:
for idx in tqdm(missing_ids):
    try:
        kegg_con = KEGG()
        kegg_entry = kegg_con.parse(kegg_con.get(idx))

        compound = pcp.get_compounds(kegg_entry['NAME'][0],'name')[0]
        
        # if compound.molecular_formula == kegg_entry['FORMULA']:
        kegg2smiles[idx] = compound.isomeric_smiles
    except:
        continue
    
    # time.sleep(3)

100%|███████████████████████████████████████████| 68/68 [04:02<00:00,  3.57s/it]


In [252]:
for idx in tqdm(missing_ids):
    try:
        kegg_con = KEGG()
        kegg_entry = kegg_con.parse(kegg_con.get(idx))

        compound = pcp.get_compounds(kegg_entry['DBLINKS']['CAS'],'name')[0]
        
        # if compound.molecular_formula == kegg_entry['FORMULA']:
        kegg2smiles[idx] = compound.isomeric_smiles
    except:
        continue

100%|█████████████████████████████████████████| 142/142 [07:56<00:00,  3.36s/it]


In [288]:
kegg2smiles['C21010'] = 'C(CC[N+](CCCN)(CCCN)CCCN)CN'
kegg2smiles['C21593'] = 'C([C@H]([C@H](C(=O)[OH])O)O)O'
kegg2smiles['C22204'] = 'CC(C)(COP(=O)([OH])OP(=O)([OH])OC[C@@H]1[C@H]([C@H]([C@@H](O1)N2C=NC3=C(N=CN=C32)N)O)OP(=O)([OH])[OH])[C@H](C(=O)NCCC(=O)NCCSC(=O)C4=CC=CC(=C4)C(=O)[OH])O'
kegg2smiles['C20256'] = 'C/C(=C/NC(=O)N)/C(=O)[OH]'
kegg2smiles['C20773'] = 'C1=CC(=CC(=C1)C(=O)[OH])C(=O)CC[C@@H]2[C@H]([C@H]([C@@H](O2)N3C=NC4=C(N=CN=C43)N)O)O'
kegg2smiles['C21649'] = 'C([C@H]([C@@H](C(=O)[OH])O)O)O'
kegg2smiles['C22935'] = 'CCC1C=[N+]2CC[C@@H]1[C@](C3=C(CC2)C4=CC=CC=C4N3)(COC(=O)C)C(=O)OC'
kegg2smiles['C22993'] = 'CC(C)CCCCCCCCCC(=O)N[C@@H](CC1=CC=CC=C1)C(=O)[OH]'
kegg2smiles['C21068'] = 'C1=C(OC=C1COP(=O)([O-])[O-])C[NH3+]'
kegg2smiles['C20959'] = 'CC(=O)C(=O)[C@H](COP(=O)([OH])[OH])O'
kegg2smiles['C21889'] = '[C-]#[N+][C@@H](CC1=CC=C(C=C1)O)C(=O)[OH]'
kegg2smiles['C22363'] = 'C[C@@H](C(=O)OP(=O)(O)OC[C@@H]1[C@H]([C@H]([C@@H](O1)N2C=NC3=C(N=CN=C32)N)O)O)N'
kegg2smiles['C20870'] = 'CC(C)(COP(=O)(O)OP(=O)(O)OC[C@@H]1[C@H]([C@H]([C@@H](O1)N2C=NC3=C(N=CN=C32)N)O)OP(=O)(O)O)[C@H](C(=O)NCCC(=O)NCCSC(=O)CCSC)O'
kegg2smiles['C21878'] = 'C1=C(C=[N+](C=C1C(=O)S)[C@H]2[C@@H]([C@@H]([C@H](O2)COP(=O)(O)O)O)O)C(=O)O'
kegg2smiles['C22934'] = 'C/C=C\1/C=[N+]2CC[C@@H]1[C@](C3=C(CC2)C4=CC=CC=C4N3)(COC(=O)C)C(=O)OC'
kegg2smiles['C20749'] = 'CC(C)(COP(=O)(O)OP(=O)(O)OC[C@@H]1[C@H]([C@H]([C@@H](O1)N2C=NC3=C(N=CN=C32)N)O)OP(=O)(O)O)[C@H](C(=O)NCCC(=O)NCCSC(=O)CCC[N+](C)(C)C)O'
kegg2smiles['C21076'] = 'C[C@H](CCCC(C)C(=O)O)[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3[C@H]2CCC4=CC(=O)CC[C@]34C)C'
kegg2smiles['C21877'] = 'C1=C(C=[N+](C=C1C(=O)OP(=O)([O-])OC[C@@H]2[C@H]([C@H]([C@@H](O2)N3C=NC4=C(N=CN=C43)N)O)O)[C@H]5[C@@H]([C@@H]([C@H](O5)COP(=O)([O-])[O-])O)O)C(=O)[O-]'
kegg2smiles['C22493'] = 'C1=CC(=C[N+](=C1)[C@H]2[C@@H]([C@@H]([C@H](O2)COP(=O)(O)OP(=O)(O)OC[C@@H]3[C@H]([C@H]([C@@H](O3)N4C=NC5=C(N=CN=C54)N)O)OP(=O)(O)O)O)O)C(=O)N'


In [291]:
pathways['Reactant'] = pathways['Reactant'].replace(kegg2smiles)
pathways['Product'] = pathways['Product'].replace(kegg2smiles)
pathways = pathways[~pathways['Reactant'].isin(missing_ids)]
pathways = pathways[~pathways['Product'].isin(missing_ids)]
pathways.to_csv('kegg_pathways_smiles.csv', index=False)

In [292]:
pathways

,Reactant,Product
0,N[C@@H](Cc1ccc(O)cc1)C(O)=O,Oc1ccccc1
1,N[C@@H](Cc1ccc(O)cc1)C(O)=O,CC(=O)OCC(CCN1C=NC2=CN=C(N=C21)N)COC(=O)C
2,N[C@@H](Cc1ccc(O)cc1)C(O)=O,[H]N([H])[H]
3,[H]O[H],Oc1ccccc1
4,[H]O[H],CC(=O)OCC(CCN1C=NC2=CN=C(N=C21)N)COC(=O)C
...,...,...
9545,[O-][As](=O)([O-])[O-],NC(=O)C1=CN(C=CC1)[C@@H]1O[C@H](COP(O)(=O)OP(O...
9546,NC(=O)c1ccc[n+](c1)[C@@H]1O[C@H](COP(O)(=O)OP(...,C([C@H](C(=O)O[As](=O)([O-])[O-])O)OP(=O)(O)O
9547,C1=CC=C(C=C1)C(COC(=O)N)COC(=O)N,CCC(=O)O[C@H]1CC[C@@H]2[C@@]1(CC[C@H]3[C@H]2CC...
9548,CNC(=O)C1=CC(=CC=C1)NC(=O)C2=CC(=C(C=C2)OC)OC,O[C@@H]1[C@@H](COP(O)(=O)OP(O)(O)=O)O[C@H]([C@...
